# Managed Spot Training and Checkpointing for built-in XGBoost

This notebook shows usage of SageMaker Managed Spot infrastructure for XGBoost training. Below we show how Spot instances can be used for the 'algorithm mode' training method with the XGBoost container. 

[Managed Spot Training](https://docs.aws.amazon.com/sagemaker/latest/dg/model-managed-spot-training.html) uses Amazon EC2 Spot instance to run training jobs instead of on-demand instances. You can specify which training jobs use spot instances and a stopping condition that specifies how long Amazon SageMaker waits for a job to run using Amazon EC2 Spot instances.

## Prerequisites
Ensuring the latest sagemaker sdk is installed. For a major version upgrade, there might be some apis that may get deprecated.

In [ ]:
!pip install sagemaker -U

### Setup variables and define functions

In [ ]:
%%time

import io
import os
import boto3
import sagemaker
import urllib

role = sagemaker.get_execution_role()
region = boto3.Session().region_name

# S3 bucket for saving code and model artifacts.
# Feel free to specify a different bucket here if you wish.
bucket = sagemaker.Session().default_bucket()
prefix = 'sagemaker/DEMO-xgboost-spot'
# customize to your bucket where you have would like to store the data
print('SageMaker version: ' + sagemaker.__version__)

### Fetching the dataset

In [ ]:
%%time

# Load the dataset
FILE_DATA = 'abalone'
urllib.request.urlretrieve("https://www.csie.ntu.edu.tw/~cjlin/libsvmtools/datasets/regression/abalone", FILE_DATA)
sagemaker.Session().upload_data(FILE_DATA, bucket=bucket, key_prefix=prefix+'/train')

### Obtaining the latest XGBoost container
We obtain the new container by specifying the framework version (0.90-1). This version specifies the upstream XGBoost framework version (0.90) and an additional SageMaker version (1). If you have an existing XGBoost workflow based on the previous (0.72) container, this would be the only change necessary to get the same workflow working with the new container.

In [ ]:
container = sagemaker.image_uris.retrieve('xgboost', region, '1.2-1')
print(container)

### Training the XGBoost model

After setting training parameters, we kick off training, and poll for status until training is completed, which in this example, takes few minutes.

To run our training script on SageMaker, we construct a sagemaker.xgboost.estimator.XGBoost estimator, which accepts several constructor arguments:

* __entry_point__: The path to the Python script SageMaker runs for training and prediction.
* __role__: Role ARN
* __hyperparameters__: A dictionary passed to the train function as hyperparameters.
* __instance_type__ *(optional)*: The type of SageMaker instances for training. __Note__: This particular mode does not currently support training on GPU instance types.
* __sagemaker_session__ *(optional)*: The session used to train on Sagemaker.

In [ ]:
hyperparameters = {
        "max_depth":"5",
        "eta":"0.2",
        "gamma":"4",
        "min_child_weight":"6",
        "subsample":"0.7",
        "objective":"reg:squarederror",
        "num_round":"50"}

instance_type = 'ml.m5.2xlarge'
output_path = 's3://{}/{}/{}/output'.format(bucket, prefix, 'abalone-xgb')
content_type = "libsvm"

If Spot instances are used, the training job can be interrupted, causing it to take longer to start or finish. If a training job is interrupted, a checkpointed snapshot can be used to resume from a previously saved point and can save training time (and cost).

To enable checkpointing for Managed Spot Training using SageMaker XGBoost we need to configure three things: 

1. Enable the `use_spot_instances` constructor arg - a simple self-explanatory boolean. 

2. Set the `max_wait constructor` arg - this is an int arg representing the amount of time you are willing to wait for Spot infrastructure to become available. Some instance types are harder to get at Spot prices and you may have to wait longer. You are not charged for time spent waiting for Spot infrastructure to become available, you're only charged for actual compute time spent once Spot instances have been successfully procured. 

3. Setup a `checkpoint_s3_uri` constructor arg - this arg will tell SageMaker an S3 location where to save checkpoints. While not strictly necessary, checkpointing is highly recommended for Manage Spot Training jobs due to the fact that Spot instances can be interrupted with short notice and using checkpoints to resume from the last interruption ensures you don't lose any progress made before the interruption.

Feel free to toggle the `use_spot_instances` variable to see the effect of running the same job using regular (a.k.a. "On Demand") infrastructure.

Note that `max_wait` can be set if and only if `use_spot_instances` is enabled and must be greater than or equal to `max_run`.

In [ ]:
use_spot_instances = True
max_run = 3600
max_wait = 7200 if use_spot_instances else None

### Simulating Spot interruption after 50 epochs

Our training job should run on 100 epochs.

However, we will simulate a situation that after 50 epochs a spot interruption occurred.

The goal is that the checkpointing data will be copied to S3, so when there is a spot capacity available again, the training job can resume from the 50th epoch.

Note the `checkpoint_s3_uri` variable which stores the S3 URI in which to persist checkpoints that the algorithm persists (if any) during training.

In [ ]:
import time

job_name = 'DEMO-xgboost-spot-1-' + time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())
print("Training job", job_name)

checkpoint_s3_uri = ('s3://{}/{}/checkpoints/{}'.format(bucket, prefix, job_name) if use_spot_instances 
                      else None)
print("Checkpoint path:", checkpoint_s3_uri)

estimator = sagemaker.estimator.Estimator(container, 
                                          role, 
                                          hyperparameters=hyperparameters,
                                          instance_count=1, 
                                          instance_type=instance_type, 
                                          volume_size=5,         # 5 GB 
                                          output_path=output_path, 
                                          sagemaker_session=sagemaker.Session(),
                                          use_spot_instances=use_spot_instances, 
                                          max_run=max_run, 
                                          max_wait=max_wait,
                                          checkpoint_s3_uri=checkpoint_s3_uri
                                         );
train_input = sagemaker.inputs.TrainingInput(s3_data='s3://{}/{}/{}'.format(bucket, prefix, 'train'), content_type='libsvm')
estimator.fit({'train': train_input}, job_name=job_name)

### Savings
Towards the end of the job you should see two lines of output printed:

- `Training seconds: X` : This is the actual compute-time your training job spent
- `Billable seconds: Y` : This is the time you will be billed for after Spot discounting is applied.

If you enabled the `use_spot_instances`, then you should see a notable difference between `X` and `Y` signifying the cost savings you will get for having chosen Managed Spot Training. This should be reflected in an additional line:
- `Managed Spot Training savings: (1-Y/X)*100 %`

### View the job training Checkpoint configuration

We can now view the Checkpoint configuration from the training job directly in the SageMaker console.  

Log into the [SageMaker console](https://console.aws.amazon.com/sagemaker/home), choose the latest training job, and scroll down to the Checkpoint configuration section. 

Choose the S3 output path link and you'll be directed to the S3 bucket were checkpointing data is saved.

You can see there are several files there:

```python
xgboost-checkpoint.49
xgboost-checkpoint.48
xgboost-checkpoint.47
xgboost-checkpoint.46
xgboost-checkpoint.45
...
```

### Continue training after Spot capacity is resumed

Now we simulate a situation where Spot capacity is resumed.

We will start a training job again, this time with 100 epochs.

What we expect is that the tarining job will start from the 50th epoch.

This is done when training job starts. It checks the checkpoint s3 location for checkpoints data. If there are, they are copied to `/opt/ml/checkpoints` on the training conatiner.

In [ ]:
hyperparameters = {
        "max_depth":"5",
        "eta":"0.2",
        "gamma":"4",
        "min_child_weight":"6",
        "subsample":"0.7",
        "objective":"reg:squarederror",
        "num_round":"100"}

instance_type = 'ml.m5.2xlarge'
output_path = 's3://{}/{}/{}/output'.format(bucket, prefix, 'abalone-xgb')
content_type = "libsvm"

Note the `checkpoint_s3_uri` which is the Checkpoint path of the previous `spot-1` managed spot training job that was completed few seconds ago. 

From this path the checkpoint files will be taken, so the training will continue from the 50th epoch.

In [ ]:
import time

job_name = 'DEMO-xgboost-spot-2-' + time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())
print("Training job", job_name)

use_spot_instances = True
max_run = 3600
max_wait = 7200 if use_spot_instances else None

print("Checkpoint path:", checkpoint_s3_uri)   

estimator = sagemaker.estimator.Estimator(container, 
                                          role, 
                                          hyperparameters=hyperparameters,
                                          instance_count=1, 
                                          instance_type=instance_type, 
                                          volume_size=5,         # 5 GB 
                                          output_path=output_path, 
                                          sagemaker_session=sagemaker.Session(),
                                          use_spot_instances=use_spot_instances, 
                                          max_run=max_run, 
                                          max_wait=max_wait,
                                          checkpoint_s3_uri=checkpoint_s3_uri
                                         );
train_input = sagemaker.inputs.TrainingInput(s3_data='s3://{}/{}/{}'.format(bucket, prefix, 'train'), content_type='libsvm')
estimator.fit({'train': train_input}, job_name=job_name)

### View the job training Checkpoint configuration after job completed 100 epochs

We can now view the Checkpoint configuration from the training job directly in the SageMaker console.  

Log into the [SageMaker console](https://console.aws.amazon.com/sagemaker/home), choose the latest training job, and scroll down to the Checkpoint configuration section. 

Choose the S3 output path link and you'll be directed to the S3 bucket were checkpointing data is saved.

You can see there are more files there, than the previous time you checked:

```python
xgboost-checkpoint.99
xgboost-checkpoint.98
xgboost-checkpoint.97
xgboost-checkpoint.96
xgboost-checkpoint.95
...
xgboost-checkpoint.49
xgboost-checkpoint.48
xgboost-checkpoint.47
xgboost-checkpoint.46
xgboost-checkpoint.45
...
```

You'll be able to see that the dates of the first five checkpoint files (49 and below), and the second group (99 and below) are grouped together, indicating the different time where the training job was run.